# ஹோட்டல் முன்பதிவு மற்றும் முன்னுரிமை உறுப்பினர் மிடில்வேருடன்

இந்த நோட்புக் **செயல்பாடு அடிப்படையிலான மிடில்வேரை** Microsoft Agent Framework-ஐ பயன்படுத்தி விளக்குகிறது. நாங்கள் நிபந்தனை வேலைப்பாடுகளின் உதாரணத்தை அடிப்படையாகக் கொண்டு, முன்னுரிமை உறுப்பினர்களுக்கு சிறப்பு சலுகைகளை வழங்கும் மிடில்வேரின் அடுக்கு சேர்க்கிறோம்.

## நீங்கள் கற்றுக்கொள்ளப்போகிறீர்கள்:
1. **செயல்பாடு அடிப்படையிலான மிடில்வேரை**: செயல்பாட்டு முடிவுகளை தடை செய்யவும் மாற்றவும்
2. **Context Access**: `context.result` ஐ செயல்படுத்திய பிறகு படிக்கவும் மற்றும் மாற்றவும்
3. **வணிகத் தர்க்கம் செயல்படுத்தல்**: முன்னுரிமை உறுப்பினர் நன்மைகள்
4. **முடிவுகளை மீறுதல்**: பயனர் நிலை அடிப்படையில் செயல்பாட்டு முடிவுகளை மாற்றவும்
5. **அதே வேலைப்பாடு, வேறுபட்ட முடிவுகள்**: மிடில்வேரால் இயக்கப்படும் நடத்தை மாற்றங்கள்

## மிடில்வேருடன் வேலைப்பாடுகளின் கட்டமைப்பு:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## நிபந்தனை வேலைப்பாடுகளிலிருந்து முக்கிய வித்தியாசம்:

**மிடில்வேர இல்லாமல்** (14-conditional-workflow.ipynb):
- பாரிஸில் அறைகள் இல்லை → alternative_agent-க்கு வழிமாற்றம்

**மிடில்வேருடன்** (இந்த நோட்புக்):
- சாதாரண பயனர் + பாரிஸ் → அறைகள் இல்லை → alternative_agent-க்கு வழிமாற்றம்
- முன்னுரிமை பயனர் + பாரிஸ் → 🌟 மிடில்வேர overrides! → கிடைக்கிறது → booking_agent-க்கு வழிமாற்றம்

## முன்பதிவுகள்:
- Microsoft Agent Framework நிறுவப்பட்டுள்ளது
- நிபந்தனை வேலைப்பாடுகளைப் புரிந்துகொள்வது (14-conditional-workflow.ipynb ஐப் பார்க்கவும்)
- GitHub token அல்லது OpenAI API key
- மிடில்வேர முறைமைகள் பற்றிய அடிப்படை புரிதல்


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## படி 1: கட்டமைக்கப்பட்ட வெளியீடுகளுக்கான Pydantic மாடல்களை வரையறுக்கவும்

இந்த மாடல்கள் ஏஜென்ட்கள் திருப்பி அளிக்கும் **schema**-ஐ வரையறுக்கின்றன. மிடில்வேர் கிடைக்கும் முடிவை மாற்றும் போது கண்காணிக்க `priority_override` புலத்தை சேர்த்துள்ளோம்.


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## படி 2: முன்னுரிமை உறுப்பினர்கள் தரவுத்தொகுப்பை வரையறுக்கவும்

இந்த டெமோவிற்காக, நாங்கள் ஒரு முன்னுரிமை உறுப்பினர் தரவுத்தொகுப்பை உருவகப்படுத்துவோம். உற்பத்தியில், இது ஒரு உண்மையான தரவுத்தொகுப்பு அல்லது API-யை கேள்வி கேட்கும்.

**முன்னுரிமை உறுப்பினர்கள்:**
- `alice@example.com` - VIP உறுப்பினர்
- `bob@example.com` - பிரீமியம் உறுப்பினர்  
- `priority_user` - சோதனை கணக்கு


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## படி 3: ஹோட்டல் முன்பதிவு கருவியை உருவாக்கவும்

நிபந்தனையுள்ள வேலைப்பாடுகளைப் போலவே, ஆனால் இப்போது இது மிடில்வேரால் தடை செய்யப்படும்!


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## படி 4: 🌟 முன்னுரிமை சரிபார்ப்பு மிடில்வேர் உருவாக்குதல் (முக்கிய அம்சம்!)

இது இந்த நோட்புக்-இன் **முக்கிய செயல்பாடு**. மிடில்வேர்:

1. **முன்கூட்டியே தடுக்கிறது** hotel_booking செயல்பாட்டு அழைப்பை
2. `next(context)` அழைப்பதன் மூலம் செயல்பாட்டை சாதாரணமாக **நடத்துகிறது**
3. `context.result`-இல் உள்ள முடிவை **சரிபார்க்கிறது**
4. பயனர் முன்னுரிமை கொண்டவர் மற்றும் அறைகள் கிடைக்கவில்லை என்றால் முடிவை **மாற்றுகிறது**
5. **மாற்றிய முடிவை** ஏஜென்டுக்கு திருப்பி அனுப்புகிறது

**முக்கிய முறை:**
```python
async def my_middleware(context, next):
    await next(context)  # Execute function
    # Now context.result contains the function's output
    if some_condition:
        context.result = new_value  # Override!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## படி 5: வழிமாற்றத்திற்கான நிலைமைகள் செயல்பாடுகளை வரையறுக்கவும்

அதே நிலைமைகள் செயல்பாடுகள், நிபந்தனை வேலைப்பாடுகளுக்கு - அவை அமைப்பான வெளியீட்டை ஆய்வு செய்து வழிமாற்றத்தை தீர்மானிக்கின்றன.


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## படி 6: தனிப்பயன் காட்சி செயல்படுத்துபவரை உருவாக்கவும்

முந்தைய செயல்படுத்துபவரைப் போலவே - இறுதி வேலைப்பாடின் வெளியீட்டை காட்சிப்படுத்துகிறது.


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## படி 7: சூழல் மாறிகள் ஏற்றவும்

LLM கிளையண்டை (GitHub Models அல்லது OpenAI) அமைக்கவும்.


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## படி 8: மிடில்வேருடன் AI முகவர்களை உருவாக்கவும்

**முக்கிய வேறுபாடு:** availability_agent உருவாக்கும்போது, `middleware` பராமரிப்பை நாங்கள் வழங்குகிறோம்!

இதன் மூலம் priority_check_middleware ஐ முகவரின் செயல்பாட்டு அழைப்பு குழாயில் சேர்க்கிறோம்.


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## படி 9: வேலைநிலையை உருவாக்கவும்

முந்தைய வேலைநிலை அமைப்பைப் போலவே - கிடைக்கும் நிலைமையை அடிப்படையாகக் கொண்ட நிபந்தனை வழிமாற்றம்.


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## படி 10: சோதனை வழக்கு 1 - பாரிசில் சாதாரண பயனர் (மாற்றம் இல்லை)

ஒரு சாதாரண பயனர் பாரிசில் அறை بک کرنے முயற்சிக்கிறார் → அறைகள் இல்லை → மாற்று முகவருக்கு வழிமாற்றம்


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## படி 11: சோதனை வழக்கு 2 - 🌟 முன்னுரிமை பயனர் பாரிஸில் (மாற்றத்துடன்!)

முன்னுரிமை உறுப்பினர் பாரிஸில் அறை بک کرنے முயற்சிக்கிறார் → ஆரம்பத்தில் அறைகள் இல்லை → 🌟 மிடில்வேரின் மாற்றம்! → booking_agent-க்கு வழிமாற்றம்

**இது மிடில்வேரின் சக்தியை காட்டும் முக்கியமான விளக்கம்!**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## படி 12: சோதனை வழக்கு 3 - முன்னுரிமை பயனர் ஸ்டாக்ஹோம் (ஏற்கனவே கிடைக்கிறது)

முன்னுரிமை பயனர் ஸ்டாக்ஹோம் முயற்சிக்கிறார் → அறைகள் கிடைக்கின்றன → மேலெழுத்து தேவையில்லை → booking_agent-க்கு வழிமாற்றம் செய்யப்படுகிறது

இது மிடில்வேர்கள் தேவையான போது மட்டுமே செயல்படுவதை காட்டுகிறது!


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## முக்கிய குறிப்புகள் மற்றும் மிடில்வேர் கருத்துக்கள்

### ✅ நீங்கள் கற்றுக்கொண்டது:

#### **1. செயல்பாடு அடிப்படையிலான மிடில்வேர் முறை**

மிடில்வேர் ஒரு எளிய async செயல்பாட்டைப் பயன்படுத்தி function calls-ஐ தடை செய்கிறது:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Before function execution
    print("Intercepting...")
    
    # Execute the function
    await next(context)
    
    # After function execution - inspect result
    if context.result:
        # Modify result if needed
        context.result = modified_value
```

#### **2. Context அணுகல் மற்றும் முடிவை மாற்றுதல்**

- `context.function` - அழைக்கப்படும் function-ஐ அணுகவும்
- `context.arguments` - function arguments-ஐ படிக்கவும்
- `context.kwargs` - கூடுதல் அளவுருக்களை அணுகவும்
- `await next(context)` - function-ஐ செயல்படுத்தவும்
- `context.result` - function-ஐ முடிவை படிக்க/மாற்றவும்

#### **3. வணிகத் தத்துவம் செயல்படுத்தல்**

எங்கள் மிடில்வேர் முன்னுரிமை உறுப்பினர் நன்மைகளை செயல்படுத்துகிறது:
- **வழக்கமான பயனர்கள்**: எந்த மாற்றமும் இல்லை, வழக்கமான workflow
- **முன்னுரிமை பயனர்கள்**: "கிடைக்கவில்லை" → "கிடைக்கிறது" என்று மாற்றம்
- **நிபந்தனைத் தத்துவம்**: தேவையான போது மட்டுமே மாற்றம் செய்கிறது

#### **4. ஒரே workflow, வேறுபட்ட முடிவுகள்**

மிடில்வேர்-இன் சக்தி:
- ✅ workflow அமைப்பில் எந்த மாற்றமும் இல்லை
- ✅ tool function-இல் எந்த மாற்றமும் இல்லை
- ✅ நிபந்தனை வழிமுறையில் எந்த மாற்றமும் இல்லை
- ✅ மிடில்வேர் மட்டும் → வேறுபட்ட நடத்தை!

### 🚀 உண்மையான உலக பயன்பாடுகள்:

1. **VIP/பிரீமியம் அம்சங்கள்**
   - பிரீமியம் பயனர்களுக்கு rate limits-ஐ override செய்யவும்
   - வளங்களுக்கு முன்னுரிமை அணுகலை வழங்கவும்
   - பிரீமியம் அம்சங்களை தன்னியக்கமாக திறக்கவும்

2. **A/B சோதனை**
   - பயனர்களை வேறுபட்ட செயல்பாடுகளுக்கு வழிமாற்றவும்
   - குறிப்பிட்ட பயனர்களுடன் புதிய அம்சங்களை சோதிக்கவும்
   - அம்சங்களை تدريجي طور پر வெளியிடவும்

3. **பாதுகாப்பு மற்றும் இணக்கம்**
   - function calls-ஐ audit செய்யவும்
   - நெகிழ்ச்சியான செயல்பாடுகளை தடுக்கவும்
   - வணிக விதிகளை அமல்படுத்தவும்

4. **செயல்திறன் மேம்பாடு**
   - குறிப்பிட்ட பயனர்களுக்கு முடிவுகளை cache செய்யவும்
   - சிக்கலான செயல்பாடுகளை தவிர்க்கவும்
   - தன்னியக்க வள ஒதுக்கீடு

5. **பிழை கையாளுதல் மற்றும் மீண்டும் முயற்சி**
   - பிழைகளை நன்கு கையாளவும்
   - retry logic-ஐ செயல்படுத்தவும்
   - மாற்று செயல்பாடுகளுக்கு fallback செய்யவும்

6. **பதிவு மற்றும் கண்காணிப்பு**
   - function செயல்பாட்டு நேரங்களை கண்காணிக்கவும்
   - அளவுருக்கள் மற்றும் முடிவுகளை பதிவு செய்யவும்
   - பயன்பாட்டு முறைமைகளை கண்காணிக்கவும்

### 🔑 டெகரேட்டர்களிலிருந்து மிடில்வேர் வேறுபாடுகள்:

| அம்சம் | டெகரேட்டர் | மிடில்வேர் |
|---------|-----------|------------|
| **Scope** | ஒற்றை function | agent-இல் உள்ள அனைத்து functions |
| **நெகிழ்வு** | வரையறையில் நிலையானது | runtime-இல் தன்னியக்கமானது |
| **Context** | வரையறுக்கப்பட்டது | முழு agent context |
| **Composition** | பல டெகரேட்டர்கள் | மிடில்வேர் pipeline |
| **Agent-Aware** | இல்லை | ஆம் (agent state-ஐ அணுகல்) |

### 📚 மிடில்வேர் எப்போது பயன்படுத்த வேண்டும்:

✅ **மிடில்வேர் பயன்படுத்தவும்:**
- பயனர்/அமர்வு நிலை அடிப்படையில் நடத்தை மாற்ற வேண்டும்
- பல functions-க்கு தத்துவத்தைப் பயன்படுத்த வேண்டும்
- agent-இன் context-ஐ அணுக வேண்டும்
- cross-cutting கவலைகளை செயல்படுத்த (logging, auth, etc.)

❌ **மிடில்வேர் பயன்படுத்த வேண்டாம்:**
- எளிய input validation (Pydantic பயன்படுத்தவும்)
- function-சார்ந்த தத்துவம் (function-இல் வைத்திருக்கவும்)
- ஒருமுறை மாற்றங்கள் (function-ஐ மாற்றவும்)

### 🎓 மேம்பட்ட முறைகள்:

```python
# Multiple middleware (execution order matters!)
middleware=[
    logging_middleware,      # Logs first
    auth_middleware,         # Then checks auth
    cache_middleware,        # Then checks cache
    rate_limit_middleware,   # Then rate limits
    priority_check_middleware  # Finally priority check
]

# Conditional middleware execution
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modify result
    else:
        # Skip execution entirely
        context.result = cached_value
```

### 🔗 தொடர்புடைய கருத்துக்கள்:

- **Agent Middleware**: agent.run() calls-ஐ தடை செய்கிறது
- **Function Middleware**: tool function calls-ஐ தடை செய்கிறது (நாம் பயன்படுத்தியது!)
- **Middleware Pipeline**: வரிசையாக செயல்படும் மிடில்வேர் சங்கிலி
- **Context Propagation**: மிடில்வேர் சங்கிலி வழியாக நிலையை அனுப்புதல்



---

**அறிவிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையை பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கிறோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதை கவனத்தில் கொள்ளவும். அதன் சொந்த மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்களுக்கும் அல்லது தவறான விளக்கங்களுக்கும் நாங்கள் பொறுப்பல்ல.
